In [ ]:
import pandas as pd
import numpy as np

# Example structure of player_data and puck_data
# player_data: ['entityId', 'team', 'position_x', 'position_y', 'timeStamp']
# puck_data: ['position_x', 'position_y', 'timeStamp']

import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Assuming player_data and puck_data are already loaded

# Function to calculate 2D distance remains unchanged (for reference)
def calculate_2d_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Preprocessing and spatial indexing
def create_spatial_index(player_data):
    """Create a spatial index (KDTree) for player positions."""
    spatial_index = {}
    for time_stamp in player_data['timeStamp'].unique():
        current_players = player_data[player_data['timeStamp'] == time_stamp]
        positions = current_players[['position_x', 'position_y']].values
        tree = KDTree(positions)
        spatial_index[time_stamp] = (tree, current_players)
    return spatial_index

# Optimized function to find the closest player to the puck
def find_closest_player_to_puck_optimized(puck_position, spatial_index, current_players):
    tree, players = spatial_index
    _, index = tree.query(puck_position)
    closest_player_id = players.iloc[index]['entityId']
    return closest_player_id

# Modified function to find multiple closest players (for expanded analysis)
def find_multiple_closest_players(puck_position, spatial_index, current_players, n=3):
    tree, players = spatial_index
    distances, indices = tree.query(puck_position, k=n)
    closest_players = players.iloc[indices]
    return closest_players, distances

# Visualization function
def visualize_gap_control(gap_control_df, player_data, puck_data, timeStamp):
    fig, ax = plt.subplots()
    # Plot players
    for team in player_data['team'].unique():
        team_data = player_data[(player_data['timeStamp'] == timeStamp) & (player_data['team'] == team)]
        ax.scatter(team_data['position_x'], team_data['position_y'], label=f'Team {team}')
    # Plot puck
    puck_position = puck_data[puck_data['timeStamp'] == timeStamp][['position_x', 'position_y']].iloc[0]
    ax.scatter(puck_position['position_x'], puck_position['position_y'], color='k', label='Puck')
    ax.legend()
    ax.set_xlabel('Position X')
    ax.set_ylabel('Position Y')
    ax.set_title(f'Player Positions and Puck at Timestamp {timeStamp}')
    plt.show()


# Main processing loop for gap control calculation, optimized
spatial_index_by_timestamp = create_spatial_index(player_data)
gap_control_data = []

for time_stamp in puck_data['timeStamp'].unique():
    current_puck_data = puck_data[puck_data['timeStamp'] == time_stamp].iloc[0]
    puck_position = (current_puck_data['position_x'], current_puck_data['position_y'])

    if -25 <= puck_position[0] <= 25:
        tree, current_player_data = spatial_index_by_timestamp[time_stamp]
        
        player_with_puck_id = find_closest_player_to_puck_optimized(puck_position, (tree, current_player_data), current_player_data)
        player_with_puck_team = current_player_data[current_player_data['entityId'] == player_with_puck_id]['team'].iloc[0]

        opposing_players = current_player_data[current_player_data['team'] != player_with_puck_team]
        if not opposing_players.empty:
            tree_opponents, _ = create_spatial_index(opposing_players)['timeStamp']
            closest_opponent_id = find_closest_player_to_puck_optimized(puck_position, (tree_opponents, opposing_players), opposing_players)

            closest_opponent_position = (
                opposing_players[opposing_players['entityId'] == closest_opponent_id]['position_x'].iloc[0],
                opposing_players[opposing_players['entityId'] == closest_opponent_id]['position_y'].iloc[0]
            )
            gap_distance = calculate_2d_distance(puck_position, closest_opponent_position)

            gap_control_data.append({
                'timeStamp': time_stamp,
                'puck_position': puck_position,
                'player_with_puck': player_with_puck_id,
                'closest_opponent': closest_opponent_id,
                'gap_distance': gap_distance
            })

gap_control_df = pd.DataFrame(gap_control_data)

# For visualization, select a sample timestamp
sample_time_stamp = puck_data['timeStamp'].unique()[0]  # Example: first unique timestamp
visualize_gap_control(gap_control_df, player_data, puck_data, sample_time_stamp)
